In [ ]:
import cv2
import matplotlib.pyplot as plt

from random import randrange, uniform
from scipy.spatial.transform import Rotation as SciR

import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random

In [ ]:


coord_root_global = (433, 470)  # x, y
img = cv2.imread("plant.bmp", cv2.IMREAD_GRAYSCALE)

n_points_back_up = 300
img2 = img[(coord_root_global[1]-n_points_back_up):coord_root_global[1], coord_root_global[0]]
img2_jnp = jnp.asarray(img2)


def find_y_coord_min(y_column):
    return jnp.where(y_column < 100)[0][-1]
    
# %time find_y_coord_min(img2_jnp)

# %time jnp.where(img2_jnp < 100)[0][-1]

a = jnp.array(img2, dtype=int)

find_y_coord_min_jit = jit(find_y_coord_min)

find_y_coord_min(a)
find_y_coord_min_jit(a)

# find_y_coord_min_jit = jit(find_y_coord_min, static_argnums=(0,))

# np.put(img, )


# %time jnp.where(img < 100)[0][-1]

# %time find_y_coord_min_jit(img2_jnp)

# print(jax.make_jaxpr(find_y_coord_min)(img2_jnp))

# print(img2_jnp)

In [ ]:
import jax
import jax.numpy as jnp
import numpy as np
from jax import grad, jit, vmap

# Create two random arrays sampled from a uniform distribution
array1 = np.random.uniform(size=(800, 800)).astype(np.float32)
array2 = jax.random.uniform(jax.random.PRNGKey(0), (800, 800), dtype=jnp.float32) # More on PRNGKey later!
# print("Shape of ndarray: ", array1.shape)
# print("Shape of DeviceArray: ", array2.shape)

def get_dot_product(W, X):
    return jnp.dot(W, X)

dot_jit = jax.jit(get_dot_product)

%time _ = np.dot(array1, array1)
%time _ = get_dot_product(array2, array2).block_until_ready()

dot_jit(array2, array2)
%time _ = dot_jit(array2, array2).block_until_ready()

jaxpr depends on tracing. The behavior of any transformed function is dependent on the traced values. You may notice the side effect on the first run but not necessarily on the subsequent calls. Hence jaxpr isn't even bothered about the global list in this case.

In [ ]:
import jax
import jax.numpy as jnp
import numpy as np
from jax import grad, jit, vmap

squared_numbers = []

# An impure function (using a global state)
def number_squared(num):
    global squared_numbers
    print(num)
    squared = num ** 2
    squared_numbers.append(squared)
    return squared

# Compiled verison
number_squared_jit = jit(number_squared)

# Make jaxpr
print(jax.make_jaxpr(number_squared_jit)(2))
print(jax.make_jaxpr(number_squared_jit)(3))

In [182]:
import numpy as np
import jax.numpy as jnp
import jax

def f(x):  # function we're benchmarking (works in both NumPy & JAX)
  return x.T @ (x - x.mean(axis=0))

print("numpy")
x_np = np.ones((1000, 1000), dtype=np.float32)  # same as JAX default dtype
%timeit f(x_np)  # measure NumPy runtime
# print("transfer")
# %time x_jax = jax.device_put(x_np)  # measure JAX device transfer time
f_jit = jax.jit(f)
print("compile")

%time f_jit(x_jax).block_until_ready()  # measure JAX compilation time
print("compiled run")
%timeit f_jit(x_jax).block_until_ready()  # measure JAX runtime

numpy
14.2 ms ± 5.56 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
compile
CPU times: user 321 ms, sys: 644 ms, total: 965 ms
Wall time: 144 ms
compiled run
14.6 ms ± 110 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
